In [1]:
import xgboost as xgb
import dask.dataframe as dd
import pandas as pd

In [2]:
data_path = '../Data/gee_features.csv'
label_path = '../Data/training_label.csv'
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [4]:
# read first 1000 rows of dataset
data = pd.read_csv(data_path, nrows=1000, usecols = lambda x: x not in 
                     ['new_ind', 'index', 'ADM1DHS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1NAME', 
                      'ADM1SALBCO', 'ADM1SALBNA', 'ADM1SALNA', 'ALT_DEM', 'ALT_GPS', 'CCFIPS',
                     'DATUM', 'F21', 'F22', 'F23', 'ZONECO', 'ZONENA', 'DHSCLUST', 'DHSREGCO', 'SOURCE',
                     'key1', 'key2', 'key3'], dtype={'DHSID': 'str'}).set_index('DHSID')

data

,DHSCC,DHSREGNA,DHSYEAR,LATNUM,LONGNUM,URBAN_RURA,elevation@AHN/AHN2_05M_INT,elevation_max@AHN/AHN2_05M_INT,elevation_min@AHN/AHN2_05M_INT,elevation@AHN/AHN2_05M_NON,...,motorized_only_friction_surface_20000,motorized_only_travel_time_20000,walking_only_friction_surface_50000,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,AL,central,2008,40.822650,19.838322,R,NaN,NaN,NaN,NaN,...,0.011207,16.502903,0.031108,226.565900,0.025174,52.417587,0.043573,216.058010,0.014725,47.410877
AL200800000002,AL,central,2008,40.696846,20.007555,R,NaN,NaN,NaN,NaN,...,0.010277,34.382580,0.035934,225.833650,0.029437,56.164467,0.039097,200.149830,0.012322,38.932705
AL200800000003,AL,central,2008,40.750040,19.974262,R,NaN,NaN,NaN,NaN,...,0.010316,27.699300,0.032862,225.121170,0.026694,52.411550,0.039717,214.108180,0.013806,43.192800
AL200800000004,AL,central,2008,40.798930,19.863338,R,NaN,NaN,NaN,NaN,...,0.010585,16.992321,0.031864,233.255630,0.025747,53.518450,0.043162,216.461870,0.014495,46.634487
AL200800000005,AL,central,2008,40.746124,19.843885,R,NaN,NaN,NaN,NaN,...,0.009745,20.305431,0.034980,249.495010,0.027163,58.282570,0.043852,206.457570,0.012748,43.546497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL201700000546,AL,Shkoder,2017,42.158990,19.541424,R,NaN,NaN,NaN,NaN,...,0.003242,11.127212,0.035101,126.969770,0.005710,20.263634,0.035503,143.897640,0.005431,22.061940
AL201700000547,AL,Shkoder,2017,42.189434,19.458715,R,NaN,NaN,NaN,NaN,...,0.006723,11.981846,0.039043,108.748070,0.004699,17.234740,0.038725,141.108540,0.004999,21.160340
AL201700000548,AL,Shkoder,2017,42.383934,19.485674,R,NaN,NaN,NaN,NaN,...,0.013820,38.764430,0.028905,100.958080,0.008513,18.662565,0.037813,127.512924,0.005551,20.148610


In [5]:
labels = pd.read_csv(label_path, usecols = pred_cols + ['DHSID'], dtype = {'Stunted_Rate':'float64'}).set_index('DHSID')
labels.head()

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000001,24.12,25.28,50.00,9.68,100.00,20.0
AL200800000002,23.04,21.98,7.69,8.33,66.67,0.0
AL200800000004,26.74,26.57,7.69,2.86,100.00,0.0
AL200800000006,27.58,28.08,0.00,9.52,NaN,0.0
AL200800000010,24.23,23.77,20.00,23.81,50.00,0.0


In [7]:
train = labels.join(data, how='inner')
train

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate,DHSCC,DHSREGNA,DHSYEAR,LATNUM,...,motorized_only_friction_surface_20000,motorized_only_travel_time_20000,walking_only_friction_surface_50000,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,24.12,25.28,50.00,9.68,100.00,20.00,AL,central,2008,40.822650,...,0.011207,16.502903,0.031108,226.565900,0.025174,52.417587,0.043573,216.058010,0.014725,47.410877
AL200800000002,23.04,21.98,7.69,8.33,66.67,0.00,AL,central,2008,40.696846,...,0.010277,34.382580,0.035934,225.833650,0.029437,56.164467,0.039097,200.149830,0.012322,38.932705
AL200800000004,26.74,26.57,7.69,2.86,100.00,0.00,AL,central,2008,40.798930,...,0.010585,16.992321,0.031864,233.255630,0.025747,53.518450,0.043162,216.461870,0.014495,46.634487
AL200800000006,27.58,28.08,0.00,9.52,NaN,0.00,AL,central,2008,40.711350,...,0.010784,31.314370,0.035164,238.875820,0.027898,56.702140,0.041174,206.234470,0.012576,40.695145
AL200800000010,24.23,23.77,20.00,23.81,50.00,0.00,AL,central,2008,40.698520,...,0.011004,32.749485,0.035676,237.618790,0.028071,57.455227,0.040892,202.150770,0.012138,39.469723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL201700000546,23.43,22.76,33.33,0.00,66.67,33.33,AL,Shkoder,2017,42.158990,...,0.003242,11.127212,0.035101,126.969770,0.005710,20.263634,0.035503,143.897640,0.005431,22.061940
AL201700000547,23.53,22.67,0.00,0.00,100.00,0.00,AL,Shkoder,2017,42.189434,...,0.006723,11.981846,0.039043,108.748070,0.004699,17.234740,0.038725,141.108540,0.004999,21.160340
AL201700000548,25.33,25.07,0.00,0.00,NaN,NaN,AL,Shkoder,2017,42.383934,...,0.013820,38.764430,0.028905,100.958080,0.008513,18.662565,0.037813,127.512924,0.005551,20.148610


In [10]:
train = train[~train.index.duplicated(keep='first')]
train

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate,DHSCC,DHSREGNA,DHSYEAR,LATNUM,...,motorized_only_friction_surface_20000,motorized_only_travel_time_20000,walking_only_friction_surface_50000,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,24.12,25.28,50.00,9.68,100.00,20.00,AL,central,2008,40.822650,...,0.011207,16.502903,0.031108,226.565900,0.025174,52.417587,0.043573,216.058010,0.014725,47.410877
AL200800000002,23.04,21.98,7.69,8.33,66.67,0.00,AL,central,2008,40.696846,...,0.010277,34.382580,0.035934,225.833650,0.029437,56.164467,0.039097,200.149830,0.012322,38.932705
AL200800000004,26.74,26.57,7.69,2.86,100.00,0.00,AL,central,2008,40.798930,...,0.010585,16.992321,0.031864,233.255630,0.025747,53.518450,0.043162,216.461870,0.014495,46.634487
AL200800000006,27.58,28.08,0.00,9.52,NaN,0.00,AL,central,2008,40.711350,...,0.010784,31.314370,0.035164,238.875820,0.027898,56.702140,0.041174,206.234470,0.012576,40.695145
AL200800000010,24.23,23.77,20.00,23.81,50.00,0.00,AL,central,2008,40.698520,...,0.011004,32.749485,0.035676,237.618790,0.028071,57.455227,0.040892,202.150770,0.012138,39.469723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL201700000546,23.43,22.76,33.33,0.00,66.67,33.33,AL,Shkoder,2017,42.158990,...,0.003242,11.127212,0.035101,126.969770,0.005710,20.263634,0.035503,143.897640,0.005431,22.061940
AL201700000547,23.53,22.67,0.00,0.00,100.00,0.00,AL,Shkoder,2017,42.189434,...,0.006723,11.981846,0.039043,108.748070,0.004699,17.234740,0.038725,141.108540,0.004999,21.160340
AL201700000548,25.33,25.07,0.00,0.00,NaN,NaN,AL,Shkoder,2017,42.383934,...,0.013820,38.764430,0.028905,100.958080,0.008513,18.662565,0.037813,127.512924,0.005551,20.148610


In [15]:
train['DHSCC'] = train['DHSCC'].astype('category')
train['DHSREGNA'] = train['DHSREGNA'].astype('category')
train['URBAN_RURA'] = train['URBAN_RURA'].astype('category')

In [18]:
model = xgb.XGBRegressor(enable_categorical=True)
model.fit(train.drop(pred_cols, axis=1), train[pred_cols])

XGBoostError: [00:39:03] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/data/data.cc:461: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001597e1bc8 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000159866234 xgboost::MetaInfo::SetInfoFromHost(xgboost::GenericParameter const&, xgboost::StringView, xgboost::Json) + 3948
  [bt] (2) 3   libxgboost.dylib                    0x000000015986516c xgboost::MetaInfo::SetInfo(xgboost::GenericParameter const&, xgboost::StringView, xgboost::StringView) + 164
  [bt] (3) 4   libxgboost.dylib                    0x00000001597f6800 XGDMatrixSetInfoFromInterface + 224
  [bt] (4) 5   libffi.8.dylib                      0x00000001010b404c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib                      0x00000001010b17d4 ffi_call_int + 1336
  [bt]